In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
import requests_cache
import lxml.html as lxl
import re
import time

In [28]:
table_makes = pd.read_html('https://www.cargurus.com/Cars/price-trends/')
makes_df = table_makes[0]
makes_df.drop(columns={'Unnamed: 0'}, inplace=True)
makes_df.head()

,Make,Avg Price,Last 30 Days,Last 90 Days,YoY
0,CarGurus Index,"$30,767",-0.07%,+0.21%,+19.04%
1,Body Styles,Body Styles,Body Styles,Body Styles,Body Styles
2,Minivan,"$24,094",-0.29%,-0.02%,+27.02%
3,Crossover,"$27,548",-0.38%,-0.58%,+19.10%
4,Van,"$34,416",+0.61%,+5.83%,+43.67%


In [29]:
# obtain data from Price Trends main page 
url1 = 'https://www.cargurus.com/Cars/price-trends/'
response = requests.get(url1)
soup = BeautifulSoup(response.text, 'lxml')

# find all links 
links = soup.find_all('a')

# create list of links to pages for each model of vehicle
makes_list = []
for i in links:
    b = i['href']
    makes_list.append(b)
    
start_position = makes_list.index('https://www.cargurus.com/') + 2
end_position = makes_list.index('https://www.bbb.org/us/ma/cambridge/profile/auto-\
listing-service/cargurus-inc-0021-139181#sealclick')

makes_links_list = []
for i in makes_list[start_position : end_position]:
    link = 'https://www.cargurus.com/Cars/price-trends/' + i
    makes_links_list.append(link)

makes_links_list[0:5]

['https://www.cargurus.com/Cars/price-trends/Acura-m4',
 'https://www.cargurus.com/Cars/price-trends/Alfa-Romeo-m124',
 'https://www.cargurus.com/Cars/price-trends/Aston-Martin-m110',
 'https://www.cargurus.com/Cars/price-trends/Audi-m19',
 'https://www.cargurus.com/Cars/price-trends/Bentley-m20']

In [30]:
%%time
frames_list1 = []

for i in range(0, len(makes_links_list)):
    table = pd.read_html(makes_links_list[i])[0]
    table.drop(columns={'Unnamed: 0'}, inplace=True)
    frames_list1.append(table)

all_models = pd.concat(frames_list1)
all_models = all_models[all_models['Make'] != 'CarGurus Index']
print('DataFrame dimensions', all_models.shape)

DataFrame dimensions (654, 5)
CPU times: user 1.21 s, sys: 111 ms, total: 1.32 s
Wall time: 31.1 s


In [31]:
all_models.rename(columns={'Make': 'Model'}, inplace=True)

In [33]:
all_models.head()

,Model,Avg Price,Last 30 Days,Last 90 Days,YoY
1,Acura,"$29,115",+0.27%,+3.09%,+19.46%
2,Acura ILX,"$23,575",+0.04%,+1.65%,+19.97%
3,Acura MDX,"$31,357",-0.87%,+0.43%,+15.26%
4,Acura RDX,"$32,320",-0.71%,+0.08%,+19.49%
5,Acura TL,"$12,259",+0.70%,+5.32%,+16.81%


#### next we will iterate through all years and models to add them to our DataFrame

In [7]:
%%time

# obtain data from Price Trends main page 

for i in makes_links_list:
    response = requests.get(i)
    soup = BeautifulSoup(response.text, 'lxml')

    # find all links 
    links = soup.find_all('a')

    # create list of links to pages for each model of vehicle 
    models_list = []
    for i in links:
        b = i['href']
        models_list.append(b)
        
    start_position = models_list.index('https://www.cargurus.com/') + 4
    end_position = models_list.index('https://www.bbb.org/us/ma/cambridge/profile/auto-listing-service/cargurus-inc-0021-139181#sealclick')

    models_links_list = []
    for i in models_list[start_position : end_position]:
        link = 'https://www.cargurus.com/Cars/price-trends/' + i
        models_links_list.append(link)

    for i in range(0, len(models_links_list)):
        table = pd.read_html(models_links_list[i])[0]
        table.drop(columns={'Unnamed: 0'}, inplace=True)
        frames_list1.append(table)

CPU times: user 16.6 s, sys: 1.34 s, total: 18 s
Wall time: 9min 4s


In [2]:
all_models2 = pd.concat(frames_list1)
all_models2 = all_models2[all_models2['Make'] != 'CarGurus Index']
all_models2.drop_duplicates(subset='Make', keep='first', inplace=True)

In [8]:
all_models2.head()

,Make,Avg Price,Last 30 Days,Last 90 Days,YoY
1,Acura,"$29,115",+0.27%,+3.09%,+19.46%
2,Acura ILX,"$23,575",+0.04%,+1.65%,+19.97%
3,Acura MDX,"$31,357",-0.87%,+0.43%,+15.26%
4,Acura RDX,"$32,320",-0.71%,+0.08%,+19.49%
5,Acura TL,"$12,259",+0.70%,+5.32%,+16.81%


## skip to this section to go faster and jump past the web scraping sections

In [11]:
# all_models2.to_excel("CarGurus_data_20220508.xlsx", index=False)

In [34]:
all_models2 = pd.read_excel("CarGurus_data_20220508.xlsx")

In [37]:
# split year and model into separate columns
def year(x):
    return x[0:4]

all_models2['model_year'] = all_models2['Make'].apply(year)

def make_model(x):
    a = x[5:].upper()
    return a

all_models2['make_model'] = all_models2['Make'].apply(make_model)

# drop rows from that are non-numeric in the model_year column
all_models2 = all_models2[all_models2.model_year.apply(lambda x: x.isnumeric())]

In [38]:
all_models2.head()

,Make,Avg Price,Last 30 Days,Last 90 Days,YoY,model_year,make_model
652,2013 Acura ILX,"$15,313",+0.61%,+3.59%,+21.48%,2013,ACURA ILX
653,2014 Acura ILX,"$15,808",-0.55%,-0.13%,+13.42%,2014,ACURA ILX
654,2015 Acura ILX,"$17,663",+0.84%,+3.10%,+18.73%,2015,ACURA ILX
655,2016 Acura ILX,"$19,796",-0.11%,+0.02%,+17.50%,2016,ACURA ILX
656,2017 Acura ILX,"$21,626",+0.31%,-0.03%,+16.24%,2017,ACURA ILX


#### next we add a column with vehicle fuel efficiency by model - it will need to include a note that older years may have lower efficiency but the guide is still valid for modelling (the fuel efficiency data is only current data, not for all years) 

#### SOURCE: https://fueleconomy.gov/feg/download.shtml

In [82]:
efficiency_table = pd.read_excel("vehicle_efficiency_data2022.xlsx")

In [83]:
# UPDATE NAMES OF SPECIFIC MODELS SO THAT WE HAVE THE MPG DATA FOR THEM 

F150_index = list(efficiency_table[efficiency_table['Model'] == 'FORD F150'].index)
for i in F150_index:
    efficiency_table.at[i, 'Model'] = 'FORD F-150'

In [84]:
# set model names to all caps so that they match model names in the CarGurus data 
def caps(x):
    return x.upper()
efficiency_table['Model'] = efficiency_table['Model'].apply(caps)

In [85]:
efficiency_table.head(3)

,Model,Displ,Cyl,Trans,Drive,Fuel,Cert Region,Stnd,Stnd Description,Underhood ID,Veh Class,Air Pollution Score,City MPG,Hwy MPG,Cmb MPG,Greenhouse Gas Score,SmartWay,Comb CO2
0,ACURA ILX,2.4,4.0,AMS-8,2WD,Gasoline,CA,L3ULEV125,California LEV-III ULEV125,NHNXV02.4KH3,small car,3,24,34,28,6,No,316
1,ACURA ILX,2.4,4.0,AMS-8,2WD,Gasoline,FA,T3B125,Federal Tier 3 Bin 125,NHNXV02.4KH3,small car,3,24,34,28,6,No,316
2,ACURA MDX,3.5,6.0,SemiAuto-10,2WD,Gasoline,CA,L3ULEV70,California LEV-III ULEV70,NHNXT03.5CCC,small SUV,5,19,26,22,5,No,411


In [88]:
# create dictionary with model names as keys and combined MPG as the values 
comb_mpg = list(efficiency_table['Cmb MPG'])
make_model = list(efficiency_table['Model'])
mpg_dict = dict(zip(make_model, comb_mpg))

all_models2['Comb_MPG'] = all_models2['make_model'].map(mpg_dict)

In [101]:
all_models2.to_excel("test3.xlsx", index=False)

In [89]:
# UPDATE MPG FOR SPECIFIC COMMON MODELS

silverados1 = list(all_models2[all_models2['make_model'] == 'CHEVROLET SILVERADO 1500'].index)
for i in silverados1:
    all_models2.at[i, 'Comb_MPG'] = 15
    
rams1 = list(all_models2[all_models2['make_model'] == 'DODGE RAM 1500'].index)
for i in rams1:
    all_models2.at[i, 'Comb_MPG'] = 21
    
accords1 = list(all_models2[all_models2['make_model'] == 'HONDA ACCORD COUPE'].index)
for i in accords1:
    all_models2.at[i, 'Comb_MPG'] = 35
    
civics1 = list(all_models2[all_models2['make_model'] == 'HONDA CIVIC'].index)
for i in civics1:
    all_models2.at[i, 'Comb_MPG'] = 33

jettas1 = list(all_models2[all_models2['make_model'] == 'VOLKSWAGEN JETTA GLI'].index)
for i in jettas1:
    all_models2.at[i, 'Comb_MPG'] = 34

pacificas1 = list(all_models2[all_models2['make_model'] == 'CHRYSLER PACIFICA HYBRID'].index)
for i in pacificas1:
    all_models2.at[i, 'Comb_MPG'] = 30

prius1 = list(all_models2[all_models2['make_model'] == 'TOYOTA PRIUS PRIME'].index)
for i in prius1:
    all_models2.at[i, 'Comb_MPG'] = 54
    
RAV1 = list(all_models2[all_models2['make_model'] == 'TOYOTA RAV4 PRIME'].index)
for i in RAV1:
    all_models2.at[i, 'Comb_MPG'] = 38

#### vehicle models where the name will need to be changed manually include:

- Chevrolet Silverado - partially complete
- Ford F150 - complete
- Dodge Ram - partially complete
- Ford Expedition 
- Honda Accord - partially complete
- Honda Civic - partially complete
- Toyota Prius models - partially complete 
- various Volkswagen models - partially complete

#### (assuming that MPG data is in the efficiency table for these models)

#### once that's complete we might want to drop rows that still have NaN values in the CombMGP  column 

In [92]:
all_models2.head()

,Make,Avg Price,Last 30 Days,Last 90 Days,YoY,model_year,make_model,Comb_MPG
652,2013 Acura ILX,"$15,313",+0.61%,+3.59%,+21.48%,2013,ACURA ILX,28
653,2014 Acura ILX,"$15,808",-0.55%,-0.13%,+13.42%,2014,ACURA ILX,28
654,2015 Acura ILX,"$17,663",+0.84%,+3.10%,+18.73%,2015,ACURA ILX,28
655,2016 Acura ILX,"$19,796",-0.11%,+0.02%,+17.50%,2016,ACURA ILX,28
656,2017 Acura ILX,"$21,626",+0.31%,-0.03%,+16.24%,2017,ACURA ILX,28


In [16]:
all_models2.shape

(6167, 8)

In [93]:
# MAY NEED TO DELETE THIS
# all_models2.dropna(axis=0,  inplace=True)
# all_models2['Comb_MPG'] = all_models2['Comb_MPG'].astype(int)

In [82]:
# all_models2.to_excel("CarGurus_data_withMPG.xlsx", index=False)

#### the code below is same as the cells above but applied to "all_models" DataFrame rather than "all_models2" where the "all_models2" data has cars by years and the other is only by model

In [94]:
# set model names to all caps so that they match model names in the CarGurus data 
def caps(x):
    return x.upper()
all_models['Model'] = all_models['Model'].apply(caps)

In [95]:
comb_mpg = list(efficiency_table['Cmb MPG'])
make_model = list(efficiency_table['Model'])
mpg_dict = dict(zip(make_model, comb_mpg))

all_models['Comb_MPG'] = all_models['Model'].map(mpg_dict)

In [96]:
# UPDATE MPG FOR SPECIFIC COMMON MODELS

silverados1 = list(all_models[all_models['Model'] == 'CHEVROLET SILVERADO 1500'].index)
for i in silverados1:
    all_models.at[i, 'Comb_MPG'] = 15
    
rams1 = list(all_models[all_models['Model'] == 'DODGE RAM 1500'].index)
for i in rams1:
    all_models.at[i, 'Comb_MPG'] = 21
    
accords1 = list(all_models[all_models['Model'] == 'HONDA ACCORD COUPE'].index)
for i in accords1:
    all_models.at[i, 'Comb_MPG'] = 35
    
civics1 = list(all_models[all_models['Model'] == 'HONDA CIVIC'].index)
for i in civics1:
    all_models.at[i, 'Comb_MPG'] = 33

jettas1 = list(all_models[all_models['Model'] == 'VOLKSWAGEN JETTA GLI'].index)
for i in jettas1:
    all_models.at[i, 'Comb_MPG'] = 34

pacificas1 = list(all_models[all_models['Model'] == 'CHRYSLER PACIFICA HYBRID'].index)
for i in pacificas1:
    all_models.at[i, 'Comb_MPG'] = 30

prius1 = list(all_models[all_models['Model'] == 'TOYOTA PRIUS PRIME'].index)
for i in prius1:
    all_models.at[i, 'Comb_MPG'] = 54
    
RAV1 = list(all_models[all_models['Model'] == 'TOYOTA RAV4 PRIME'].index)
for i in RAV1:
    all_models.at[i, 'Comb_MPG'] = 38

In [99]:
all_models.tail(25)

,Model,Avg Price,Last 30 Days,Last 90 Days,YoY,Comb_MPG
15,TOYOTA FJ CRUISER,"$25,011",-0.68%,-3.02%,+2.00%,34
16,TOYOTA HIGHLANDER,"$32,614",-1.11%,-2.09%,+17.35%,23
17,TOYOTA HIGHLANDER HYBRID,"$37,866",+0.25%,-1.40%,+34.93%,35
21,TOYOTA PRIUS,"$17,270",+1.69%,+5.20%,+39.23%,49
23,TOYOTA PRIUS PRIME,"$31,395",+4.19%,+7.52%,+35.31%,54
25,TOYOTA RAV4,"$26,680",-0.32%,-1.39%,+23.58%,29
26,TOYOTA RAV4 HYBRID,"$35,918",+1.67%,+0.81%,+26.43%,40
27,TOYOTA RAV4 PRIME,"$55,403",+7.84%,+9.61%,+19.29%,38
28,TOYOTA SEQUOIA,"$33,921",-2.09%,+0.26%,+26.57%,14
29,TOYOTA SIENNA,"$26,366",-0.14%,-0.74%,+29.18%,15


In [78]:
#all_models['Avg Price'] = all_models['Avg Price'].astype(int)
# all_models['Last 30 Days'] = all_models['Last 30 Days'].astype(float)
# all_models['Last 90 Days'] = all_models['Last 90 Days'].astype(float)
# all_models['YoY'] = all_models['YoY'].astype(float)
all_models.dropna(axis=0,  inplace=True)
all_models['Comb_MPG'] = all_models['Comb_MPG'].astype(int)

In [80]:
all_models.to_excel("MPG_by_model_V2.xlsx", index=False)